In [1]:
import pandas as pd
from cow_amm_trade_envy.models import Contracts
from cow_amm_trade_envy.datasources import get_transaction_receipt
from tqdm import tqdm

def analyze_surplus(file_path, pool_address):

    pool_address = pool_address.lower()

    def isin_pool(topic):
        return pool_address in topic.hex()

    def log_is_used(log):
        return any(isin_pool(topic) for topic in log.topics)

    def logs_are_used(logs):
        return any(log_is_used(log) for log in logs)

    file = pd.read_csv(file_path)
    total_surplus, used_surplus = 0, 0

    for _, row in tqdm(file.iterrows()):
        if row["trade_envy"] > 0:
            tx_receipt = get_transaction_receipt(row["call_tx_hash"])
            is_used = logs_are_used(tx_receipt.logs)

            if not is_used:
                total_surplus += row["trade_envy"]
            else:
                used_surplus += row["trade_envy"]

    return {
        "total_unused_surplus": total_surplus / 10**18,
        "total_used_surplus": used_surplus / 10**18,
    }


if __name__ == "__main__":
    file_path = "data/cow_amm_missed_surplus.csv"
    pool_address = Contracts.USDC_WETH_POOL

    surplus_results = analyze_surplus(file_path, pool_address)
    print(f"Total unused surplus: {surplus_results['total_unused_surplus']}")
    print(f"Total used surplus: {surplus_results['total_used_surplus']}")


499it [00:00, 40708.73it/s]

Total unused surplus: 0.0
Total used surplus: 0.0


In [5]:
get_transaction_receipt("0x5381141986041a1b42931e5e37bdab3cce7672a830914390c6da18a477ca930b")["logs"]

Fetching new transaction receipt


TypeError: Object of type AttributeDict is not JSON serializable